In [55]:
import pandas as pd
import numpy as np

## Чтение файлов

In [2]:
articles = pd.read_csv('articles.csv', sep=';', header=None)
stopwords = pd.read_csv('stopwords.csv', sep=';', header=None)

In [4]:
articles.head()

,0,1
0,1,Bradley Charles Cooper born January 5 1975 is ...
1,2,Cooper enrolled in the MFA program at the Acto...
2,3,Cooper found greater success with the romantic...
3,4,Labeled a sex symbol by the media Cooper was n...
4,5,Cooper was born on January 5 1975 in Abingto...


In [5]:
stopwords.head()

,0
0,x
1,y
2,your
3,yours
4,yourself


## Обработка данных

- привести все слова к нижнему регистру

In [10]:
articles['lower'] = articles[1].apply(lambda x: x.lower())

- отбросить все символы, которые не являются латинскими буквами _(и, вероятно, пробелами)_

In [15]:
articles['latin'] = articles['lower'].apply(lambda text: ''.join([symbol for symbol in text if 'a' <= symbol <= 'z' or symbol.isspace()]))

- удалить все стоп-слова из articles с помощью таблицы stopwords

In [30]:
articles['cleared'] = articles['latin'].apply(lambda text: [word for word in text.split() if (stopwords[0] == word).sum() == 0])

# Шаг 2

In [97]:
def NPMI(text):
    global words_freq
    global words_total
    global pairs_freq
    global pairs_total
    
    # абсолютные частотности слов 
    words_total += len(text)
    for word in text:
        words_freq[word] = words_freq.get(word, 0) + 1
    
    # составим пары слов
    pairs = list(zip(text, text[1:]))
    pairs = list(map(lambda x: x[0] + " " + x[1], pairs))
    
    pairs_total += len(pairs)
    for pair in pairs:
        pairs_freq[pair] = pairs_freq.get(pair, 0) + 1 

In [98]:
words_freq = {}
words_total = 0

pairs_freq = {}
pairs_total = 0

articles['NPMI'] = articles['cleared'].apply(lambda text: NPMI(text))

In [99]:
# вероятности слов, P(a), P(b)
for key in words_freq:
    words_freq[key] /= words_total
    
# вероятности пар слов, P(a, b)
for key in pairs_freq:
    pairs_freq[key] /= pairs_total
    
# теперь для каждой пары слов PMI
PMI = {}
for key in pairs_freq:
    word1, word2 = key.split()

    PMI_ab = np.log(pairs_freq[key] / (words_freq[word1] * words_freq[word2]))
    PMI[key] = PMI_ab

# теперь для каждой пары слов NPMI 
NPMI = {}
for key in PMI:
    NPMI[key] = -(PMI[key] / np.log(pairs_freq[key]))
    
print(NPMI)
#     return NPMI

{'bradley charles': 0.7077920357649904, 'charles cooper': 0.4448664415623883, 'cooper born': 0.38845952516473986, 'born january': 0.7503317969863372, 'january american': 0.5003619687715339, 'american actor': 0.3165772248741097, 'actor filmmaker': 0.6782838473516152, 'filmmaker recipient': 0.8915767796624147, 'recipient accolades': 0.8204791355588148, 'accolades including': 0.5913746260244988, 'including british': 0.4973886528036473, 'british academy': 0.6043795038841797, 'academy film': 0.32636781488895455, 'film award': 0.23206618803551027, 'award grammy': 0.5842470039994894, 'grammy awards': 0.6135867543150647, 'awards addition': 0.48305296030297284, 'addition nominations': 0.5415750122280776, 'nominations academy': 0.569122878528677, 'academy awards': 0.5975023941054345, 'awards golden': 0.6135867543150647, 'golden globe': 0.9603093502806682, 'globe awards': 0.6135867543150647, 'awards tony': 0.6337192541997552, 'tony award': 0.784983420895518, 'award cooper': 0.17705622419849962, '

In [100]:
sorted(NPMI.items(), key=lambda x: x[1], reverse=True)[:50]

[('fish fry', 1.0052239644191399),
 ('linings playbook', 1.0051069538755188),
 ('nightmare alley', 1.0051069538755188),
 ('los angeles', 1.0051069538755188),
 ('guardians galaxy', 1.0049706878472697),
 ('licorice pizza', 1.004805384273404),
 ('barack obama', 1.004805384273404),
 ('willy wanker', 1.004805384273404),
 ('iberian peninsula', 1.004805384273404),
 ('irina shayk', 1.0045902343009177),
 ('phil wenneck', 1.0045902343009177),
 ('bipolar disorder', 1.0045902343009177),
 ('severely deformed', 1.0045902343009177),
 ('clint eastwood', 1.0045902343009177),
 ('pounds kg', 1.0045902343009177),
 ('parke custis', 1.0045902343009177),
 ('amos bronson', 1.0045902343009177),
 ('allison rader', 1.0045902343009177),
 ('steven moll', 1.0045902343009177),
 ('bing crosby', 1.0045902343009177),
 ('ella fitzgerald', 1.0045902343009177),
 ('joel whitburns', 1.0045902343009177),
 ('rabbit foot', 1.0045902343009177),
 ('foot minstrels', 1.0045902343009177),
 ('elks rendezvous', 1.0045902343009177),
 